In [53]:
from __future__ import division
from code.organize import *
from code.roughEDA import *
from code.survey_processor import *

import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
sv = survey('data/survey/')

In [59]:
sv.cat_processor()

In [64]:
sv.num_processor()

In [65]:
df = sv.fin_data

In [66]:
df.shape

(4504, 55)

In [67]:
sv.raw_data.shape

(13826, 54)

In [32]:
sv.num_processor()

In [33]:
df_n = sv.fin_data

In [171]:
with open('data/survey/GSS.sps') as f:
    content = f.readlines()

In [199]:
for c in content:
    if c.strip()=='VALUE LABELS':
        ind = content.index(c)+1
lis = []
for i in xrange(ind,len(content)-2):
    if content[i].strip().strip('/').strip()[0].isalpha()==True:
        lis.append(i)
        

In [261]:
x = [(lis[i],lis[i+1]) for i in xrange(len(lis)-1) ]
x.append((lis[-1],len(content)-3))

In [422]:
from collections import defaultdict
d3 = {}
for k,v in d.iteritems():
    d2 = defaultdict(int)
    for i in v:
        text = " ".join(i.strip().split()[1:])
        d2[re.findall('"([^"]*)"', text)[0]] = int(i.strip().split()[0])
        d3[k] = d2

In [454]:
d4 = {}
for i in x:
    a = content[i[0]:i[1]]
    d4[a[0].strip().strip("/").strip().lower()] = a[1:]

In [8]:
s = sv.survey_cols

In [18]:
dictn = extr_val_labels('data/survey/GSS.sps')

In [10]:
for col in s[:-1]:
    df_r[col] = df_r[col].apply(lambda x: dictn[col][x])

In [26]:
sv.num_process().shape

(4504, 55)

In [27]:
df_r.shape

(4504, 55)